In [1]:
import sys
sys.path.append('/home/ilia/pythia8313-install/lib/')

import pickle
import gzip
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool, GATConv
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
from itertools import product
from pythia8 import Pythia
import fastjet
from safe_root import ROOT

In [2]:
# Определяем структуру данных
ROOT.gInterpreter.Declare('''
struct EventData {
    Int_t event_id;
    Float_t Tar;
    std::vector<Float_t> px;
    std::vector<Float_t> py;
    std::vector<Float_t> pz;
    std::vector<Float_t> e;
    Int_t nParticles;
};
''')

True

In [ ]:
# Настройки генерации
pTHatMin_values = [0, 2, 3, 4, 5]
nEvent = 10000000
etaMax = 5.0
minPT = 0.25

for pTHatMin in pTHatMin_values:

    print(f"Генерация pTHatMin = {pTHatMin}...")

    # Создаем отдельный файл для каждого pTHatMin
    file_name = f"pythia_pTHatMin_{pTHatMin}.root"
    root_file = ROOT.TFile(file_name, "RECREATE")
    
    # Создаем TTree
    tree = ROOT.TTree("events", f"Events for pTHatMin={pTHatMin}")
    event_data = ROOT.EventData()
    tree.Branch("event", event_data)
    
    pythia = Pythia()
    pythia.readString("Beams:idA = 2212")
    pythia.readString("Beams:idB = 2212")
    pythia.readString("Beams:eCM = 27.")
    pythia.readString("HardQCD:all = on")
    pythia.readString(f"PhaseSpace:pTHatMin = {pTHatMin}")
    pythia.init()

    for i in range(nEvent):
        if not pythia.next(): continue

        if i % 1000 == 0:
            print(f"pTHatMin={pTHatMin}: События - {i}", end='\r')
        
        # Очищаем векторы для нового события
        event_data.px.clear()
        event_data.py.clear()
        event_data.pz.clear()
        event_data.e.clear()

        Tar = None
        for p in pythia.event:
            if abs(p.status()) == 23:
                Tar = p.pT()
                break
        if Tar < 1: continue
        
        for p in pythia.event:
            if not p.isFinal(): continue
            if not p.isVisible(): continue
            if abs(p.eta()) > etaMax: continue
            if p.pT() < minPT: continue

            event_data.px.push_back(p.px())
            event_data.py.push_back(p.py())
            event_data.pz.push_back(p.pz())
            event_data.e.push_back(p.e())

        event_data.event_id = i  
        event_data.Tar = Tar
        event_data.nParticles = len(event_data.px)
        tree.Fill()

        
root_file.Write()
root_file.Close()